In [1]:
import torch
import numpy as np
from copy import deepcopy

## Analyse results

In [2]:
def read_file(kernel_type, dim, benchmark_index, nruns):
    file_name = 'tsp_botorch_'+kernel_type+'_EI_dim_'+str(dim)+'benchmark_index_'+str(benchmark_index)+'_nrun_'+str(nruns)+'.pkl'
    data = torch.load(file_name, weights_only=False)
    l = data['outputs']
    l = [float(i) for i in l]
    return l


def read_file_filename(file_name):
    data = torch.load(file_name, weights_only=False)
    l = data['outputs']
    l = [float(i) for i in l]
    return l


def read_file_no_anchor(kernel_type, dim, benchmark_index, nruns):
    file_name = 'tsp_botorch_'+kernel_type+'_EI_dim_'+str(dim)+'benchmark_index_no_anchor_'+str(benchmark_index)+'_nrun_'+str(nruns)+'.pkl'
    data = torch.load(file_name, weights_only=False)
    l = data['outputs']
    l = [float(i) for i in l]
    return l

In [25]:
import os

def analyse_trial(dim=10, benchmark_index=0, cut=200):
    folders = os.listdir('./results')
    nruns = 20
    results_dict = {}

    for folder in folders:
        if '.' in folder:
            continue
        results_dict[folder] = []
        for nrun in range(nruns):
            results_dict[folder].append(read_file_filename(os.path.join('./results', folder, folder+f'_nrun_{nrun}.pkl')))

    all_results = []
    for key in results_dict.keys():
        results_dict[key] = np.array(results_dict[key])
        all_results.append(results_dict[key][:, :cut])
    # print(all_results[0].shape)
    # return all_results
    global_minimum = np.min(all_results)
    print(global_minimum)
    best_so_far = [np.minimum.accumulate(res, axis=1) for res in all_results]
    regrets = [bfs - global_minimum for bfs in best_so_far]
    for i, key in enumerate(results_dict.keys()):
        results_dict[key] = regrets[i]
        # results_dict[key] = all_results[i]
    return results_dict

In [23]:
import numpy as np
import pandas as pd
from sklearn.metrics import auc

import numpy as np
import pandas as pd
from sklearn.metrics import auc

import numpy as np
import pandas as pd
from sklearn.metrics import auc

def evaluate_algorithms(r: dict, f_opt=None, threshold=None):
    """
    r: dict of {algorithm_name: np.ndarray of shape (n_repeats, n_iterations)}
    f_opt: known global minimum value (float)
    threshold: optional regret threshold to measure how many iterations are needed
    
    Returns:
        pd.DataFrame with aggregated metrics for each algorithm
    """
    results = []

    for algo, regrets in r.items():
        regrets = np.array(regrets)  # shape: (n_repeats, n_iterations)
        n_repeats, n_iterations = regrets.shape

        best_so_far = np.minimum.accumulate(regrets, axis=1)  # shape: (n_repeats, n_iterations)
        final_best = best_so_far[:, -1]
        auc_vals = np.array([
            auc(np.arange(1, n_iterations+1), best_so_far[i])
            for i in range(n_repeats)
        ])

        metrics = {
            "algorithm": algo,
            "final_best_mean": np.mean(final_best),
            "final_best_std": np.std(final_best),
            "auc_best_so_far_mean": np.mean(auc_vals),
            "auc_best_so_far_std": np.std(auc_vals),
        }

        if f_opt is not None:
            simple_regrets = regrets - f_opt  # ✅ 对于最小化，目标值应减去最优值
            cumulative_regrets = np.cumsum(simple_regrets, axis=1)
            mean_simple = np.mean(simple_regrets, axis=1)
            final_simple = simple_regrets[:, -1]
            final_cum = cumulative_regrets[:, -1]

            metrics.update({
                "mean_simple_regret_mean": np.mean(mean_simple),
                "mean_simple_regret_std": np.std(mean_simple),
                "final_simple_regret_mean": np.mean(final_simple),
                "final_simple_regret_std": np.std(final_simple),
                "cumulative_regret_mean": np.mean(final_cum),
                "cumulative_regret_std": np.std(final_cum),
            })
        else:
            metrics.update({
                "mean_simple_regret_mean": None,
                "mean_simple_regret_std": None,
                "final_simple_regret_mean": None,
                "final_simple_regret_std": None,
                "cumulative_regret_mean": None,
                "cumulative_regret_std": None,
            })

        if threshold is not None:
            evals_to_threshold = []
            for i in range(n_repeats):
                for j in range(n_iterations):
                    if best_so_far[i, j] <= threshold:
                        evals_to_threshold.append(j + 1)
                        break
                else:
                    evals_to_threshold.append(n_iterations)
            evals_to_threshold = np.array(evals_to_threshold)
            metrics.update({
                "evals_to_threshold_mean": np.mean(evals_to_threshold),
                "evals_to_threshold_std": np.std(evals_to_threshold),
            })
        else:
            metrics.update({
                "evals_to_threshold_mean": None,
                "evals_to_threshold_std": None,
            })

        results.append(metrics)

    return pd.DataFrame(results)


r = analyse_trial(cut=200)
k = evaluate_algorithms(r, 329.969018)
k

329.969018


,algorithm,final_best_mean,final_best_std,auc_best_so_far_mean,auc_best_so_far_std,mean_simple_regret_mean,mean_simple_regret_std,final_simple_regret_mean,final_simple_regret_std,cumulative_regret_mean,cumulative_regret_std,evals_to_threshold_mean,evals_to_threshold_std
0,tsp_botorch_merge_EI_dim_10benchmark_index_k5_...,330.052889,0.174432,66132.302860,146.084482,7.123681,1.101528,5.588660,12.780406,1424.736140,220.305690,None,None
1,tsp_botorch_merge_EI_dim_10benchmark_index_wit...,330.007728,0.071842,66165.393498,156.162093,8.125217,0.896046,3.862375,8.188660,1625.043384,179.209159,None,None
2,tsp_botorch_merge_EI_dim_10benchmark_index_k4_...,330.014179,0.169592,66076.026011,121.981277,6.789502,0.954584,1.329030,1.470682,1357.900358,190.916777,None,None
3,tsp_botorch_merge_EI_dim_10benchmark_index_shi...,329.981921,0.038710,66064.740460,115.012314,6.806800,0.898856,1.618742,2.085492,1361.360041,179.771187,None,None
4,tsp_botorch_merge_EI_dim_10benchmark_index_no_...,330.046437,0.125101,66191.435457,162.856862,9.331795,4.754941,3.069438,3.990725,1866.358929,950.988251,None,None
5,tsp_botorch_merge_EI_dim_10benchmark_index_k4_...,329.988373,0.046074,66074.662285,107.345022,6.668632,0.878565,1.206449,0.828497,1333.726416,175.712997,None,None
6,tsp_botorch_merge_EI_dim_10benchmark_index_k3_...,329.981921,0.038710,66099.460076,129.746168,8.211784,1.081205,3.062376,4.991346,1642.356888,216.240960,None,None
7,tsp_botorch_merge_EI_dim_10benchmark_index_k4_...,330.059340,0.071842,66034.827592,94.798764,10.309546,1.320541,7.071004,7.905608,2061.909211,264.108260,None,None
8,tsp_botorch_merge_EI_dim_10benchmark_index_leh...,329.988373,0.046074,66081.411928,118.031734,6.796995,0.784716,3.655009,10.745716,1359.399008,156.943292,None,None
9,tsp_botorch_merge_EI_dim_10benchmark_index_k34...,329.981921,0.038710,66069.695340,134.290461,6.761151,0.859748,1.502918,1.502509,1352.230265,171.949612,None,None


In [26]:
k['algorithm'][1]

'tsp_botorch_merge_EI_dim_10benchmark_index_with_half_pairwise_0'

In [28]:
r = analyse_trial()

329.969018


In [29]:
for key in r.keys():
    print(f'{key} mean regret: ', r[key].mean())

tsp_botorch_merge_EI_dim_10benchmark_index_k5_with_permutation_pattern_0 mean regret:  2.4405371559999964
tsp_botorch_merge_EI_dim_10benchmark_index_with_half_pairwise_0 mean regret:  2.6058774399999947
tsp_botorch_merge_EI_dim_10benchmark_index_k4_with_permutation_pattern_0 mean regret:  2.159056133999994
tsp_botorch_merge_EI_dim_10benchmark_index_shift_pairwise_pattern_0 mean regret:  2.1025477379999966
tsp_botorch_merge_EI_dim_10benchmark_index_no_anchor_0 mean regret:  2.7361840119999923
tsp_botorch_merge_EI_dim_10benchmark_index_k4_pairwise_pattern_0 mean regret:  2.152172987999995
tsp_botorch_merge_EI_dim_10benchmark_index_k3_with_permutation_pattern_0 mean regret:  2.2761458159999948
tsp_botorch_merge_EI_dim_10benchmark_index_k4_permutation_pattern_0 mean regret:  1.9531769419999947
tsp_botorch_merge_EI_dim_10benchmark_index_lehmer_pairwise_pattern_0 mean regret:  2.1859212059999944
tsp_botorch_merge_EI_dim_10benchmark_index_k34_with_permutation_pattern_0 mean regret:  2.1273221

In [527]:
r['tsp_botorch_merge_EI_dim_10benchmark_index_k4_with_permutation_pattern_0'][0]

array([33.001712, 33.001712, 22.943312, 22.943312, 22.943312, 15.213584,
       15.213584, 15.213584, 15.213584, 15.213584, 15.213584, 15.213584,
       15.213584, 15.213584, 15.213584, 15.213584, 15.213584, 12.891008,
       12.891008, 12.891008, 12.891008, 12.891008, 12.891008, 12.891008,
        3.741928,  3.741928,  3.741928,  3.741928,  3.741928,  3.741928,
        3.741928,  3.741928,  3.741928,  3.741928,  3.741928,  3.096768,
        3.096768,  3.096768,  3.096768,  1.548384,  1.548384,  1.548384,
        1.548384,  1.548384,  1.548384,  1.548384,  1.548384,  1.548384,
        1.548384,  1.548384,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064, 

In [465]:
r['tsp_botorch_merge_EI_dim_10benchmark_index_k5_with_hash_bucket_0'][8]

array([42.531792, 42.531792, 19.078448, 19.078448, 19.078448, 19.078448,
       11.735816, 11.735816, 11.735816, 11.735816, 11.735816, 11.735816,
       11.735816, 11.735816, 11.735816, 11.735816, 11.735816, 11.735816,
       11.735816, 11.735816, 11.735816, 10.187432,  8.12292 ,  8.12292 ,
        5.935472,  3.612896,  3.612896,  3.612896,  3.612896,  2.322576,
        2.322576,  2.322576,  2.322576,  2.322576,  2.322576,  2.322576,
        2.322576,  1.548384,  1.548384,  0.387096,  0.387096,  0.387096,
        0.387096,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032, 

In [467]:
r['tsp_botorch_mallows_EI_dim_10benchmark_index_0'][7]

array([24.1046  , 24.1046  , 24.1046  , 24.1046  ,  7.612888,  7.612888,
        7.612888,  7.612888,  7.612888,  7.612888,  7.612888,  7.612888,
        2.322576,  2.322576,  2.322576,  2.322576,  2.322576,  2.322576,
        2.322576,  2.322576,  2.322576,  2.322576,  1.548384,  1.29032 ,
        1.29032 ,  1.29032 ,  1.29032 ,  1.29032 ,  1.29032 ,  1.29032 ,
        1.29032 ,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  0.64516 ,  0.64516 ,  0.64516 ,
        0.64516 ,  0.516128,  0.516128,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032, 

In [151]:
r['tsp_botorch_bitonic_EI_dim_10benchmark_index_0'].mean()

np.float64(5.3665869299999756)

In [152]:
r['tsp_botorch_mallows_EI_dim_10benchmark_index_0'].mean()

np.float64(2.239048861999994)

In [167]:
r['tsp_botorch_mallows_EI_dim_10benchmark_index_0'][15]

array([68.307712, 56.06796 , 56.06796 , 14.304264, 14.304264, 14.304264,
       14.304264, 14.304264, 14.304264, 14.304264, 12.37488 , 12.37488 ,
       12.37488 , 12.37488 , 12.37488 , 12.37488 , 12.37488 , 12.37488 ,
       12.37488 , 12.37488 , 12.116816, 10.955528, 10.955528, 10.4394  ,
       10.4394  ,  6.18744 ,  1.161288,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      , 

In [534]:
from itertools import permutations
import numpy as np

def get_relative_order(window):
    order = sorted(range(len(window)), key=lambda i: window[i])
    relative = [0] * len(window)
    for rank, idx in enumerate(order):
        relative[idx] = rank
    return tuple(relative)

def pattern_count_feature_circular(perm, k):
    """
    计算循环滑动窗口的排序模式计数。
    
    参数：
      - perm: 输入置换（长度为 n 的 list 或 np.array）
      - k: 滑动窗口长度

    返回：
      - 长度为 k! 的 numpy 向量，统计每种模式出现的次数
    """
    n = len(perm)
    patterns = list(permutations(range(k)))
    pattern_to_index = {p: i for i, p in enumerate(patterns)}
    counts = np.zeros(len(patterns), dtype=int)

    for i in range(n):
        window = [perm[(i + j) % n] for j in range(k)]  # 循环索引
        pat = get_relative_order(window)
        idx = pattern_to_index[pat]
        counts[idx] += 1

    return counts

def right_multiply(x, anchor):
    """
    对置换 x 进行右乘 anchor，返回 x ◦ anchor。
    
    参数：
      - x: list[int]，长度为 n 的置换，例如 [2, 0, 1]
      - anchor: list[int]，长度为 n 的置换，例如 [1, 2, 0]
    
    返回：
      - result: list[int]，x 右乘 anchor 的结果
    """
    return [anchor[xi] for xi in x]

In [543]:
perm1 = [3, 1, 4, 2, 0]
perm2 = [2, 4, 1, 0, 3]

k = 3
feat1 = pattern_count_feature_circular(perm1, k)
feat2 = pattern_count_feature_circular(perm2, k)
print(feat1-feat2)

[ 0  1  0 -1  0  0]


In [559]:
anchor = [3, 4, 0, 1, 2]
right1 = right_multiply(perm1, anchor)
right2 = right_multiply(perm2, anchor)
feat1 = pattern_count_feature_circular(right1, k)
feat2 = pattern_count_feature_circular(right2, k)
print(feat1-feat2)

[-1  1  1  0  0 -1]


In [566]:
perm = [3, 1, 4, 2, 0]
k =3 

n = len(perm)
patterns = list(permutations(range(k)))
pattern_to_index = {p: i for i, p in enumerate(patterns)}
counts = np.zeros(len(patterns), dtype=int)

for i in range(n):
    window = [perm[(i + j) % n] for j in range(k)]  # 循环索引
    print(window)
    pat = get_relative_order(window)
    idx = pattern_to_index[pat]
    counts[idx] += 1

[3, 1, 4]
[1, 4, 2]
[4, 2, 0]
[2, 0, 3]
[0, 3, 1]


In [565]:
patterns

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [560]:
right1

[1, 4, 2, 0, 3]

In [556]:
right2

[0, 2, 4, 3, 1]